# Section I: Basic Array Operations

* To enter REPL, type `julia` in your terminal (PATH variable in shell should be set correctly).
* To get help in REPL, type `?` to enter `API` mode.
* To install a package, type `]` to enter `Pkg` mode. Type `?<Enter>` if you want some help in `Pkg` mode.
* To run shell command in REPL, type `;` to enter shell mode.
* To input `≈`, type `\approx<TAB>`, see full list of unicode input of julia [here](https://docs.julialang.org/en/v1/manual/unicode-input/).
* `@test` means stop program and throw and exception whenever a clause returns `false`. "function" start with `@` character is called a [macro](https://docs.julialang.org/en/v1/manual/metaprogramming/).

In [37]:
# Julia Matrix Operation is fast. To know how fast it is, you need a benchmark package. Type `]` and `add BenchmarkTools<Enter>` in your REPL.
using BenchmarkTools, Test

cnot = [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]  # a matrix
b = [1, 1im, true, 0.4e2] # a vector
c = [1, 1im, 0.4e2, true] # a vector

# run tests
@test cnot*b ≈ c
@test_throws DimensionMismatch cnot*randn(3)  # expected to raise specific error

# run a benchmark
res1 = @benchmark $cnot*$b  # `$` sign means evaluation first, used in `@benchmark` to avoid taking time to evaluate cnot input account.

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  1
  --------------
  minimum time:     61.977 ns (0.00% GC)
  median time:      63.230 ns (0.00% GC)
  mean time:        92.478 ns (18.80% GC)
  maximum time:     64.490 μs (99.82% GC)
  --------------
  samples:          10000
  evals/sample:     981

# Can it be faster? Yes if the matrix is small and static!

For static array, we can avoid all allocations.

### **Used in**
* Quantum Circuit Simulation

### **Challenge!**
Read the document of package `StaticArrays`
https://github.com/JuliaArrays/StaticArrays.jl
and show it is really fast!

In [35]:
# install `StaticArrays`
# sa = <some static matrix> 
# sb = <some static vector> 

using StaticArrays: SMatrix, SVector
scnot = SMatrix{4, 4}(cnot)
sb = SVector{4}(b)


@testset "static arrays" begin
    @test scnot*sb ≈ c
    using Statistics: median
    res2 = @benchmark $scnot*$sb
    @test median(res2).time < median(res1).time / 5  # at least 5 times faster
end

Test Summary: | Pass  Total
static arrays |    2      2


Test.DefaultTestSet("static arrays", Any[], 2, false)

# Linear Algebra

See [docs](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/).

### **Used in**
* Quantum Monte Carlo
* Non-Interacting fermionic systems (include mean field)

### **Challenge!**
Try to figure out how to use get eigenvalues, singular values and qr and pass the test

In [17]:
using LinearAlgebra: eigen, svd, qr, I, det, tr
# hint, here you probabily want to type `?` and `eigen<Enter>` to get help in an REPL.

# sv_cnot = <get the singular value of matrix cnot>
# ev_cnot = <get the eigenvalue of matrix cnot>
# q_cnot = <get the Q matrix from QR decomposition of cnot>
# det_cnot = <get the determinant of matrix cnot>
# tr_cnot = <get the trace of matrix cnot>
sv_cnot = svd(cnot).S
ev_cnot = eigen(cnot).values
q_cnot = qr(cnot).Q
det_cnot = det(cnot)
tr_cnot = tr(cnot)

@testset "linalg" begin
    @test sv_cnot ≈ ones(4)
    @test ev_cnot ≈ [-1, 1, 1, 1]
    @test q_cnot*q_cnot' ≈ I
    @test det_cnot == -1
    @test tr_cnot == 2
end

Test Summary: | Pass  Total
linalg        |    5      5


Test.DefaultTestSet("linalg", Any[], 5, false)

# Large sparse matrix eigensolver

[KrylovKit](https://github.com/Jutho/KrylovKit.jl) is a package for solving large sparse matrix.


### **Used in**
* Exact diagonalization
* Cluster pertubation theory (CPT)
* Numerical Renomalization Group

### **Challenge!**
Get the lowest singular value of specific sparse matrix

In [23]:
using SparseArrays: SparseMatrixCSC, sparse, nnz
sp = kron(SparseMatrixCSC(cnot), sparse(I, 100, 100))

# install `KrylovKit`
# ev = <get lowest eigenvalue of sp>
using KrylovKit: eigsolve
vals = eigsolve(sp, 1, :SR)[1][1]

@testset "sparse" begin
    @test sp |> nnz == 100*4    # here `x |> f` is same as calling f(x).
    @test vals[1] ≈ minimum(eigen(sp |> Matrix).values)
end

Test Summary: | Pass  Total
sparse        |    2      2


Test.DefaultTestSet("sparse", Any[], 2, false)

# Tensor contraction

[TensorOperations](https://github.com/Jutho/TensorOperations.jl) is a high performance package for tensor contractions.

### **Used in**
* Tensor Networks (TRG, DMRG,...)


### **Challenge!**
Try to calculate the following contraction
$C_{lj} = A_{i,j,k}B_{i,k,l}$

In [26]:
A = randn(6, 10, 5)
B = randn(6, 5, 7)

# install TensorOperations, and using it
# @tensor C[a, b] := <specify contraction>

using TensorOperations
@tensor C[l,j] := A[i,j,k] * B[i,k,l]

@test C |> size == (7, 10)

Test Passed

# Save and load data

[DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/index.html) is the `txt` format save and load standard module.

[JLD2 and FileIO](https://github.com/simonster/JLD2.jl) uses HDF5 format to save and load, suited for large data file.


### **Challenge!**
Read out the saved data.

In [30]:
using DelimitedFiles

a = randn(Float64, 3,3)
writedlm("data/_test.dat", a)
# b = <read data from file `data/_test.dat`>
b = readdlm("data/_test.dat")

# FileIO
# install FileIO and JLD2
using FileIO, JLD2
jldopen("data/_example.jld2", "w") do f
    f["A"] = a
end

# b_jld2 = <read data from file `data/_example.jld2`>
b_jld2 = load("data/_example.jld2")["A"]
@testset "file reading" begin
    @test b ≈ a
    @test b_jld2 ≈ a
end

Test Summary: | Pass  Total
file reading  |    2      2


Test.DefaultTestSet("file reading", Any[], 2, false)

# `################################################################`
# Now, you can embrace the world of Notebooks!
# But not the programmer's world, this is why we have an **ADVANCED SESSION** ;D
# `################################################################`

# Defining functions

We have [multiple dispatch](https://docs.julialang.org/en/v1/manual/functions/) + [type tree](https://docs.julialang.org/en/v1/manual/types/) instead of classes in object oriented design.

In [71]:
# define function f
f(x) = x^2
function f(x::Vector{T}) where T  # `where` means we want to know the type parameter T.
    println("we get type `$T`!")
    append!(copy(x), x)
end
function f(x::Vector{<:Complex})
    append!(copy(x), x |> conj)
end
# inplace vector version of f
function f!(x::Vector)
    append!(x, x)
end
f(x::String) = x^2

@testset "functions" begin
    @test f(3) == 9
    @test f.([1,2,3,4,5]) == [1,4,9,16,25]  # broadcast
    @test f("bili") == "bilibili"
    @test f.(["bili", "dili"]) == ["bilibili", "dilidili"]

    x = [1, 2, 3]
    y = [1im, 2+3im, 3]
    @test f(x) == [1,2,3,1,2,3]  # repeat the array
    @test f(y) == [1im,2+3im,3,-1im,2-3im,3]  # repeat, but with conjugate
    y = copy(x)
    @test (f!(y); y) == [1,2,3,1,2,3]  # inplace version of f
end

we get type `Int64`!
Test Summary: | Pass  Total
functions     |    7      7


Test.DefaultTestSet("functions", Any[], 7, false)

# Type Stability

[Type stability](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Avoid-changing-the-type-of-a-variable-1) is the hardest and most important part of writing high performance Julia programs.

### **Challenge!**
Fix the allocation below to increase the performance of calculating Fibonacci.

In [70]:
# fix Fibonacci to improve performance
function fib(n)
    a = 0
    b = 1
    for i = 1:n-1
        c = a + b
        a = b
        b = c
    end
    b
end
display(@benchmark fib(30))
display(@code_warntype fib(30))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.034 ns (0.00% GC)
  median time:      2.044 ns (0.00% GC)
  mean time:        2.101 ns (0.00% GC)
  maximum time:     82.409 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

nothing

Body::Int64
│╻     -5  1 ── %1  = (Base.sub_int)(n, 1)::Int64
││╻╷╷╷  Type   │    %2  = (Base.sle_int)(1, %1)::Bool
│││╻     unitrange_last   │          (Base.sub_int)(%1, 1)
││││     │    %4  = (Base.ifelse)(%2, %1, 0)::Int64
││╻╷╷   isempty   │    %5  = (Base.slt_int)(%4, 1)::Bool
││       └───       goto #3 if not %5
││       2 ──       goto #4
││       3 ──       goto #4
│        4 ┄─ %9  = φ (#2 => true, #3 => false)::Bool
│        │    %10 = φ (#3 => 1)::Int64
│        │    %11 = (Base.not_int)(%9)::Bool
│        └───       goto #10 if not %11
│        5 ┄─ %13 = φ (#4 => 0, #9 => %14)::Int64
│        │    %14 = φ (#4 => 1, #9 => %16)::Int64
│        │    %15 = φ (#4 => %10, #9 => %22)::Int64
│╻     +6  │    %16 = (Base.add_int)(%13, %14)::Int64
│╻╷    iterate8  │    %17 = (%15 === %4)::Bool
││       └───       goto #7 if not %17
││       6 ──       goto #8
││╻     +   7 ── %20 = (Base.add_int)(%15, 1)::Int64
│╻     iterate   └───       goto #8
│        8 ┄─ %22 = φ (#7 => %20)::

In [68]:
@testset "fibonacci" begin
    @test fib(30) == 832040
    @test (@allocated fib(30))  == 0
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.033 ns (0.00% GC)
  median time:      2.482 ns (0.00% GC)
  mean time:        2.396 ns (0.00% GC)
  maximum time:     51.433 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

Test Summary: | Pass  Total
fibonacci     |    2      2


Test.DefaultTestSet("fibonacci", Any[], 2, false)

# Final Boss

Give data file `example.txt`, with each row a tuple of $i, j, w_{ij}$.
The problems is to find the ground state configuration of the classical Ising hamiltonian $H = \sum\limits_{i,j} w_{ij}\sigma_i \sigma_j$

This is the code chanlledge of 2016 UCAS summer school.

### Simmulated Annealling
Fix the code of simulated annealing

1. correctify the code
2. improve the performance

In [75]:
# please open `simulated_annealing.ipynb` to continue